In [ ]:
import shutil
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.resnet50 import (ResNet50,
                                                    preprocess_input as resnet50_preprocess_input,
                                                    decode_predictions as resnet50_decode_predictions)
import numpy as np
import json
from matplotlib import pyplot as plt, cm
import tensorflow as tf
import numpy as np

import os

In [ ]:
root_dir = os.getcwd()
model = ResNet50(weights='imagenet')

# Download the json file of list of classes in imagenet with index
if os.path.isfile('imagenet_class_index.json') == False:
    !wget "https://raw.githubusercontent.com/raghakot/keras-vis/master/resources/imagenet_class_index.json"

CLASS_INDEX = json.load(open("imagenet_class_index.json"))
classlabel  = []
for i_dict in range(len(CLASS_INDEX)):
    classlabel.append(CLASS_INDEX[str(i_dict)][1])

In [ ]:
os.chdir(root_dir)
os.getcwd()

In [ ]:
# Image loading directory
load_clean = os.path.join(root_dir, '100class', 'clean_images')
load_adv = os.path.join(root_dir, '100class', 'adv_images')

#Image saving directory
save_clean = os.path.join(root_dir, '100class_robust', 'clean_images')
save_adv = os.path.join(root_dir, '100class_robust', 'adv_images')

In [ ]:
os.chdir(root_dir)
os.chdir(load_clean)

for class_name in os.listdir():
    # print(class_name)
    os.chdir(class_name)
    for file_name in os.listdir():
        print(file_name)
        cln_img = load_img(file_name, target_size=(224,224))
        cln_img = img_to_array(cln_img).astype(np.uint8)
        cln_img = np.expand_dims(cln_img, axis=0)

        tmp_img = resnet50_preprocess_input(cln_img.copy())
        img_pred = model.predict(tmp_img)
        
        prediction = resnet50_decode_predictions(img_pred, top=1)
        class_lbl = classlabel.index(prediction[0][0][1])
        class_list = [itm for val in CLASS_INDEX.values() for itm in val]
        # print(class_list.index(class_name))
        
        # choose only those images that are correctly classified
        if int(class_list.index(class_name)/2) == class_lbl:
            print('true')
            os.makedirs(os.path.join(save_clean, class_name), exist_ok=True)
            os.makedirs(os.path.join(save_adv, class_name), exist_ok=True)
            shutil.copy(os.path.join(load_clean, class_name, file_name), os.path.join(save_clean, class_name))
            shutil.copy(os.path.join(load_clean, class_name, file_name), os.path.join(save_adv, class_name))
    os.chdir('..')
os.chdir('..')
#             # For its adversarial counterpart, check if its adv image misclassifes correctly
#             adv_img = os.path.join(load_adv, class_name, file_name)
            
#             if os.path.isfile(adv_img):
#                 img = load_img(adv_img, target_size=(224,224))
#                 img = img_to_array(img).astype(np.uint8)
#                 img = np.expand_dims(img, axis=0)

#                 tmp_img = resnet50_preprocess_input(img.copy())
#                 img_pred = model.predict(tmp_img)

#                 prediction = resnet50_decode_predictions(img_pred, top=1)
#                 class_lbl = classlabel.index(prediction[0][0][1])
#                 class_list = [itm for val in CLASS_INDEX.values() for itm in val]

#                 # choose only those iamges that misclassify correctly
#                 if int(class_list.index(class_name)/2) != class_lbl:
#                     # print(class_lbl)
#                     # plt.imshow(np.uint8(cln_img[0]))
#                     # plt.show()
#                     # plt.imshow(np.uint8(img[0]))
#                     # plt.show()
#                     os.makedirs(os.path.join(save_clean, class_name), exist_ok=True)
#                     os.makedirs(os.path.join(save_adv, class_name), exist_ok=True)
                    
#                     shutil.copy(os.path.join(load_clean, class_name, file_name), os.path.join(save_clean, class_name))
#                     shutil.copy(os.path.join(load_adv, class_name, file_name), os.path.join(save_adv, class_name))
#     os.chdir('..')
# os.chdir('..')